In [1]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')
from pyspark.sql import SparkSession, Row

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-10-27 13:57:06,820 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [21]:
spark = SparkSession(sc)

input_path = "hdfs:/data/clickstream.csv"
clickstream_rdd = sc.textFile(input_path)

header = clickstream_rdd.first()  
clickstream_rdd = clickstream_rdd.filter(lambda line: line != header)  


def formatted(line):
    line = line.split("\t") 
    return {
        "user_id": int(line[0]),  
        "session_id": int(line[1]),  
        "event_type": line[2],  
        "event_page": line[3],  
        "timestamp": int(line[4]) 
    }

clickstream_formatted = clickstream_rdd.map(formatted)

In [22]:
error_sessions = clickstream_formatted.filter(lambda line: "error" in line["event_type"]) \
                                            .map(lambda line: ((line['user_id'], line["session_id"]), line["timestamp"])) \
                                            .reduceByKey(lambda a, b: min(a, b))  

error_sessions = error_sessions.collectAsMap()

def is_after_error(line):
    session_id = line["session_id"]
    user_id = line["user_id"]
    return (user_id, session_id) in error_sessions and line["timestamp"] >= error_sessions[(user_id, session_id)]

clickstream = clickstream_formatted.filter(lambda line: not is_after_error(line))

In [23]:
page_events = clickstream.filter(lambda line: line["event_type"] == "page")

In [32]:
def routes(user_id, session_id, events):
    events_sorted = sorted(events, key=lambda x: x['timestamp'])
    linked = []
    previous = None

    for event in events_sorted:
        current = event['event_page']

        if previous is None or previous != current:
            linked.append(current)
            previous = current
    linked = "-".join(linked)
    return (user_id, session_id), linked


routes_rdd = page_events.map(lambda line: ((line["user_id"], line["session_id"]), line)) \
                                           .groupByKey() \
                                           .mapValues(list) \
                                           .map(lambda x: routes(x[0][0], x[0][1], x[1]))

In [33]:
routes_rdd.first()

((2209, 541), 'main-bonus-online-internet-news-main')

In [36]:
routes_rdd = routes_rdd.map(lambda x: (x[1], 1))  

In [37]:
routes_rdd

PythonRDD[43] at RDD at PythonRDD.scala:53

In [43]:
route_counts = routes_rdd.reduceByKey(lambda a, b: a + b)
#route_counts = route_counts.collect()

In [91]:
sorted_counts_rdd = route_counts.sortBy(lambda x: x[1], ascending=False)

In [92]:
formatted_data = sorted_counts_rdd.map(lambda x: f"{x[0]}\t{x[1]}").take(30)


In [96]:
with open('text.txt', 'w') as file:
    for line in formatted_data:
        file.write(line + '\n')